# Log NASA 

#### ETL - Jose Carlos Montes 

Nota: esta practica esta basada en la solución de Arturo Sanchez, ya que no fue posible seguir las dudas antes a falta de Docker

Comenzamos Spark e importamos el archivo de access.logsmall:

In [50]:
import pandas as pd

In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [1]:
import urllib.request
f = urllib.request.urlretrieve("https://cunef.instructure.com/courses/2199/files/118341/download", "./apache.access.log_small")

In [20]:
raw_data = sc.textFile('apache.access (4).txt')

In [22]:
raw_data.take(10)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0',
 'ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713',
 'uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0" 304 0',
 'slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] "GET /history/skylab/skylab.html HTTP/1.0" 200 1687',
 'piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/launchmedium.gif HTTP/1.0" 200 11853',
 'slppp6.intermind.net - - [01/Aug/1995:00:00:11 -0400] "GET

Los registros siguen un patron de url -- [Fecha] "GET /dirección de solicitud HTTP/ un id 

Vamos a hacer un count para ver cuantos registros tenemos: 

In [23]:
raw_data.count()

3432

Vamos a realizar las formulas de parseo y mapeo de experesiones regulares para poder analizar la data:

In [24]:
import re
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1
n_logs = raw_data.count()

In [25]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [26]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])
parsed_def = parsed_rdd.map(lambda line: map_log(line))

## Minimo, maximo, y media del tamaño de las peticiones: 

Realizamos una formula para cuantificar las expresiones regulares: 

In [27]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats()

(count: 3432, mean: 16051.863636363621, stdev: 53247.8157482, max: 887988.0, min: 0.0)

### Numero de peticiones de cada codigo de respuesta:

In [43]:
parsed_def.take(2)

[('in24.inetnebr.com',
  '-',
  '-',
  '01/Aug/1995:00:00:01',
  '0400',
  'GET',
  '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  'HTTP/1.0"',
  None,
  '200',
  '1839'),
 ('uplherc.upl.com',
  '-',
  '-',
  '01/Aug/1995:00:00:07',
  '0400',
  'GET',
  '/',
  'HTTP/1.0"',
  None,
  '304',
  '0')]

In [28]:
n_codes = parsed_def.map(lambda line: (line[-2], 1)).distinct().count()
codes_count = (parsed_def.map(lambda line: (line[-2], 1))
          .reduceByKey(lambda a, b: a + b)
          .takeOrdered(n_codes, lambda x: -x[1]))
codes_count

[('200', 3140), ('304', 219), ('302', 50), ('404', 22), ('403', 1)]

In [47]:
parsed_def.map(lambda line: (line[-2], 1)).take(2)

[('200', 1), ('304', 1)]

## Mostrar 20 hosts que han sido visitados mas de 10 veces

In [29]:
result = parsed_def.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b).takeOrdered(20, lambda x: -x[1])
result

[('ix-min1-02.ix.netcom.com', 78),
 ('uplherc.upl.com', 71),
 ('port26.ts1.msstate.edu', 59),
 ('h96-158.ccnet.com', 56),
 ('in24.inetnebr.com', 55),
 ('piweba3y.prodigy.com', 54),
 ('thing1.cchem.berkeley.edu', 54),
 ('adam.tower.com.au', 44),
 ('ip55.van2.pacifier.com', 43),
 ('ppp1016.po.iijnet.or.jp', 41),
 ('hsccs_gatorbox07.unm.edu', 40),
 ('www-b2.proxy.aol.com', 40),
 ('www-d1.proxy.aol.com', 39),
 ('133.43.96.45', 37),
 ('port13.wavenet.com', 37),
 ('pc-heh.icl.dk', 33),
 ('haraway.ucet.ufl.edu', 32),
 ('193.84.66.147', 31),
 ('www-c1.proxy.aol.com', 30),
 ('143.158.26.50', 29)]

## Mostrar los 10 enpoints más visitados

In [30]:
result = parsed_def.map(lambda line: (line[6],1)).reduceByKey(lambda a, b: a + b).takeOrdered(10, lambda x: -x[1])
result

[('/images/KSC-logosmall.gif', 167),
 ('/images/NASA-logosmall.gif', 160),
 ('/images/MOSAIC-logosmall.gif', 122),
 ('/images/WORLD-logosmall.gif', 120),
 ('/images/USA-logosmall.gif', 118),
 ('/images/ksclogo-medium.gif', 106),
 ('/', 85),
 ('/history/apollo/images/apollo-logo1.gif', 74),
 ('/images/launch-logo.gif', 69),
 ('/images/ksclogosmall.gif', 66)]

## Mostrar los 10 enpoints más visitados que no tienen como respuesta 200

In [31]:
result = (parsed_def.filter(lambda line: line[9] != '200')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b)
          .takeOrdered(10, lambda x: -x[1]))
result

[('/images/NASA-logosmall.gif', 25),
 ('/images/KSC-logosmall.gif', 24),
 ('/images/MOSAIC-logosmall.gif', 17),
 ('/images/WORLD-logosmall.gif', 17),
 ('/images/USA-logosmall.gif', 16),
 ('/images/ksclogo-medium.gif', 10),
 ('/software/winvn/bluemarb.gif', 8),
 ('/software/winvn/winvn.html', 8),
 ('/images/construct.gif', 8),
 ('/software/winvn/wvsmall.gif', 6)]

## Calcular el número de hosts distintos

In [48]:
parsed_def.map(lambda line: line[0]).distinct().count()

311

## Calcular el número de host únicos cada día

In [49]:
from datetime import datetime
def day_month(line):
    date_time = line[3]
    return datetime.strptime(date_time[:11], "%d/%b/%Y") #Se parsea la fecha para trabajar con ella tal y como vimos en clase.
result = parsed_def.map(lambda line:  (day_month(line), 1)).reduceByKey(lambda a, b: a + b).distinct().collect()
result

[(datetime.datetime(1995, 8, 1, 0, 0), 3432)]

Solo tenemos un día en este file

## Calcular la media de las peticiones diarias por host

In [53]:
unique_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}

In [59]:
print(" La media de las peticiones de el dia que tenemos es de", round(day_count_dct["Media"][0],2))

 La media de las peticiones de el dia que tenemos es de 11.04


## Mostrar una lista de 40 endpoints distintos con codigo 404:

In [60]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(40, lambda x: -x[1]))
result

[('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/readme.txt', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-004/movies/', 1),
 ('/history/history.htm', 1),
 ('/www/software/winvn/winvn.html', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1)]

## Mostrar el top 25 de los enpoints con mas errores 404

In [61]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(25, lambda x: -x[1]))
result

[('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/readme.txt', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-004/movies/', 1),
 ('/history/history.htm', 1),
 ('/www/software/winvn/winvn.html', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1)]

## Top 5 de dias que se genero el error (solo tenemos un dia...)

In [62]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line:  (day_month(line), 1))
          .reduceByKey(lambda a, b: a+b).collect())
day = [x[0] for x in result]
count = [x[1] for x in result]
day_count_dct = {'day':day, 'count':count}

In [71]:
print(" El dia que se genero mayor error fue en", day_count_dct["day"][0])

 El dia que se genero mayor error fue en 1995-08-01 00:00:00
